In [1]:
import numpy as np
from scipy.stats import norm

In [5]:
def displaced_diffusion_option(S: float,
                            K: float, 
                            T: float, 
                            r: float,
                            risky_sigma: float, 
                            alpha: float, 
                            beta: float,
                            delta: list,
                            d: float,
                            option_type: str,
                            call: bool) -> float:
    """
    Calculate the price of call/put option using displaced diffusion model

    Parameters:
    S: float : current stock price
    K: float : strike price
    T: float : time to maturity
    r: float : risk-free rate
    risky_sigma: float : volatility of the risky asset
    alpha: float : proportion of firm's value in the risky asset
    beta: float : debt-equity ratio
    delta: list: dividend fraction from risky asset
    d: list: fixed dividend from riskless asset
    option_type: str : "vanilla", "digital_asset_or_nothing", "digital_cash_or_nothing"
    call: bool : True for call option, False for put option
    
    Returns:
    float : option price
    """

    dividend_adjustment = np.prod([1- d_k for d_k in delta])

    a = alpha * (1+beta) * dividend_adjustment
    b = (1-alpha-alpha*beta) * r**T - sum([d_k * r**(T-k) for k, d_k in enumerate(d,1)])

    x_numerator = np.log((a*S)/((K-b*S)*r**(-T)))
    x_denominator = risky_sigma * np.sqrt(T)
    x = x_numerator / x_denominator + 0.5*risky_sigma**2*T

    if option_type == "vanilla":
        if call:
            return a*S*norm.cdf(x) - (K-b*S)*r**(-T)*norm.cdf(x - risky_sigma*np.sqrt(T))
        else:
            return (K-b*S)*r**(-T)*norm.cdf(-x + risky_sigma*np.sqrt(T)) - a*S*norm.cdf(-x)
    
    if option_type == "digital_asset_or_nothing":
        if call:
            return S*np.exp(-r*T)*norm.cdf(x)
        else:
            return S*np.exp(-r*T)*norm.cdf(-x)
        
    if option_type == "digital_cash_or_nothing":
        if call:
            return np.exp(-r*T)*norm.cdf(x)
        else:
            return np.exp(-r*T)*norm.cdf(-x)

In [15]:
beta = 0
S = 40
delta = d = [0]
r = 1.05

alpha = 0.75
risky_sigma = 0.4
K = 30
T = 1/12

displaced_diffusion_option(S, K, T, r, risky_sigma, alpha, beta, delta, d, "vanilla", True)

10.121852156844067